# Data Report -
### Energy consumption as driver of greenhouse gas emissions and the influence of renewables.

## Main Question
The main question of this project will discuss is:

1. How does the amount of energy consumed influence the net greenhouse gas emissions of European countries?

Furthermore, the project will answer the follow-up question:

2. And how is this influenced by the share of renewables in total energy?

## Data Sources
This project utilizes three data sources from the European data portal Eurostat. Eurostat was chosen for this project because it provides high-quality, comparable data for all EU countries, not just individual nations. This consistency is crucial for a comprehensive analysis.

Eurostat's supra-national approach to data collection and publication is particularly advantageous for this project, as tackling climate change necessitates a broad, as-global-as-possible perspective. By examining data across 27 EU countries, the main question of 'how energy consumption influences greenhouse gas emissions' can be addressed in a more generalized manner.

Moreover, the significant variations in renewable energy usage across these countries allow for a robust analysis of the secondary question: 'how the correlation between energy consumption and greenhouse gas emissions is influenced by the share of renewables'. This comparative analysis across 27 countries enhances the generalizability of the findings.


## Data Pipeline

## Results and Limitations